In [1]:
import polars as pl
from test_cfg import *

In [2]:
all_df, daily_df = get_dfs()

In [ ]:
daily_pdf = daily_df.to_pandas()

In [ ]:
all_df

stock,date,time,midEnd,trade,volume
cat,date,time,f64,i32,i32
"""APD""",2019-01-02,09:30:00,157.66,0,0
"""APD""",2019-01-02,09:30:10,157.66,0,0
"""APD""",2019-01-02,09:30:20,157.66,0,0
"""APD""",2019-01-02,09:30:30,157.66,0,0
"""APD""",2019-01-02,09:30:40,157.66,0,0
…,…,…,…,…,…
"""APA""",2019-12-27,15:59:20,26.135,298,298
"""APA""",2019-12-27,15:59:30,26.155,-426,426
"""APA""",2019-12-27,15:59:40,26.125,-100,100


In [ ]:
daily_pdf.pivot(index='stock', columns='date', values='adv').T.plot()

In [ ]:
daily_df.filter(pl.col('stock') == 'ADS')

stock,date,adv,vol
cat,date,f64,f64
"""ADS""",2019-01-02,null,null
"""ADS""",2019-01-03,null,null
"""ADS""",2019-01-04,null,null
"""ADS""",2019-01-07,null,null
"""ADS""",2019-01-08,null,null
…,…,…,…
"""ADS""",2019-12-19,90423.619048,0.000303
"""ADS""",2019-12-20,88878.952381,0.000304
"""ADS""",2019-12-23,85781.4,0.000297


In [ ]:
import sys
sys.path.append('../')
from models.impact_model import *

In [ ]:
trade_df = all_df

In [ ]:
trade_df = trade_df.join(daily_df, how='left', on=['stock', 'date'])

In [ ]:
ow = OWModel(0.98, 100000)
impacted_df = ow.subtract_impact(trade_df).collect()

In [ ]:
# Is vol too low? need to multiply by 100?
impacted_df.filter(pl.col('impacted_price').is_not_null())

stock,date,time,midEnd,trade,volume,adv,vol,impact,impacted_price
cat,date,time,f64,i32,i32,f64,f64,f64,f64
"""ADSK""",2019-02-04,09:30:00,149.25,-404,404,237350.7,0.000308,-0.001071,149.248929
"""ADSK""",2019-02-04,09:30:10,149.22,0,0,237350.7,0.000308,-0.00105,149.21895
"""ADSK""",2019-02-04,09:30:20,149.22,0,0,237350.7,0.000308,-0.001029,149.218971
"""ADSK""",2019-02-04,09:30:30,149.22,0,0,237350.7,0.000308,-0.001008,149.218992
"""ADSK""",2019-02-04,09:30:40,149.22,0,0,237350.7,0.000308,-0.000988,149.219012
…,…,…,…,…,…,…,…,…,…
"""ADS""",2019-12-26,15:59:20,110.285,-85,85,81498.9,0.000286,0.002311,110.287311
"""ADS""",2019-12-26,15:59:30,110.27,-88,88,81498.9,0.000286,0.001634,110.271634
"""ADS""",2019-12-26,15:59:40,110.335,-100,100,81498.9,0.000286,0.000885,110.335885


In [ ]:
import polars as pl
df = pl.DataFrame({'a':[1, 1, 1, 2, 2], 'b': [1, 2, 3, 1, 5]})
df.with_columns(pl.col('b').update)

AttributeError: 'Expr' object has no attribute 'update'

In [ ]:
from collections import OrderedDict

In [ ]:
o = OrderedDict({'a': 1, 'b': 2})
o

OrderedDict([('a', 1), ('b', 2)])

In [ ]:
df1 = pl.DataFrame(df)
df1
df = 1
df1

a,b
i64,i64
1,1
1,2
1,3
2,1
2,5


In [1]:
!pip install "C:\Users\lachl\OneDrive\Desktop\ImpactPy\base\dist\impactpy-0.0.17.tar.gz"

Processing c:\users\lachl\onedrive\desktop\impactpy\base\dist\impactpy-0.0.17.tar.gz
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for impactpy: filename=impactpy-0.0.17-py3-none-any.whl size=6006 sha256=a234c6242a16265a8d3c29cfe1e3d90f0476a5d3dc12b81e330ddf9983ed3e04
  Stored in directory: c:\users\lachl\appdata\local\pip\cache\wheels\70\30\98\7e6e58d7e06131cd0c0d8fa03de192677d52bd7b8f10c215b5
Successfully built impactpy
  Attempting uninstall: impactpy
    Found existing installation: impactpy 0.0.17
    Uninstalling impactpy-0.0.17:
      Successfully uninstalled impactpy-0.0.17


In [2]:
import sys
sys.path.insert(0, '../')
from data.processing.data_processing import load_raw_csvs
from impactpy.data import BinSchedule, BinnedData, ColumnBindings
from datetime import date, time, timedelta, datetime
import polars as pl
pl.enable_string_cache()
from pathlib import Path

In [3]:
DATA_DIR = Path().resolve().parents[0] / 'data' / 'raw_bin_samples'
FILES: list[Path] = list(DATA_DIR.iterdir())
DTYPES = {'date': pl.Date, 'time': pl.String, 'stock': pl.Categorical,
	'midEnd': pl.Float64, 'trade': pl.Int32}

BIN_DURATION = timedelta(seconds=10)
OPEN_TIME = time(hour=9, minute=30)
CLOSE_TIME = time(hour=16)

In [4]:
lf = load_raw_csvs(FILES, DTYPES)
lf = lf.rename({'stock': 'symbol'})

In [5]:
df = lf.collect()

In [6]:
DATES: list[date] = sorted(pl.Series(df.select('date').unique()).to_list())
SYMBOLS: set[str] = set(df.get_column('symbol').cat.get_categories().to_list())

In [7]:
schedule = BinSchedule(BIN_DURATION, OPEN_TIME, CLOSE_TIME, DATES, SYMBOLS)

In [12]:
bindings = ColumnBindings(
	symbol='symbol',
	date='date',
	time='time',
	price='midEnd',
	trade='trade'
)
data = BinnedData(df.lazy(), schedule, bindings)

In [10]:
data.data.collect()

symbol,date,time,midEnd,trade
cat,date,time,f64,i32
"""ABBV""",2019-01-02,09:30:00,90.99,-100
"""ABBV""",2019-01-02,09:30:10,91.12,0
"""ABBV""",2019-01-02,09:30:20,91.38,48
"""ABBV""",2019-01-02,09:30:30,91.25,-29
"""ABBV""",2019-01-02,09:30:40,91.25,-1
…,…,…,…,…
"""AGN""",2019-12-27,15:59:20,191.435,0
"""AGN""",2019-12-27,15:59:30,191.43,200
"""AGN""",2019-12-27,15:59:40,191.41,-114
